In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib

In [1]:
!pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218921 sha256=a795a2c1ad6f3c94e39e81a0a760ea2e9eb8dc9ec7be2ad62910f29462d625fa
  Stored in directory: /root/.cache/pip/wheels/1e/5c/47/759b32beb377aee150eb4c1f1b2b7635032cc19ea09ce076fd
Successfully built pandas_ta


In [2]:
import pandas_datareader.data as pdr

def get_stock_data(code):
  df = pdr.DataReader("{}.JP".format(code), "stooq").sort_index()
  return df

In [3]:
import plotly.graph_objs as go
#import talib as ta
import pandas_ta as ta
import datetime as dt
import pandas as pd
import numpy as np

name = '積水ハウス'
code = 1928
df = get_stock_data(code)

layout = { "height": 600,
           "title"  : { "text": "{} {}".format(code, name), "x":0.5 }, 
           "xaxis" : { "rangeslider": { "visible": False } },
           "yaxis": { "title": "価格（円）", "side": "left", 
                     "tickformat": "," },
          "plot_bgcolor": "light blue" }

rdf = df[dt.datetime(2021,11,1):dt.datetime(2022,3,31)]
rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

# ローソク足
data =  [ go.Candlestick(x=rdf.index, 
                           open=rdf["Open"], high=rdf["High"], 
                           low=rdf["Low"], close=rdf["Close"],   
                           increasing_line_color="red", 
                           decreasing_line_color="gray")]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], 
                        x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()

In [12]:
!pip install TA-Lib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached TA-Lib-0.4.26.tar.gz (272 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for TA-Lib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for TA-Lib
Failed to build TA-Lib
ERROR: Could not build wheels for TA-Lib, which is required to install pyproject.toml-based projects


In [11]:
df = df.ta.cdl_pattern(name="closingmarubozu")
df

[X] Please install TA-Lib to use closingmarubozu. (pip install TA-Lib)


,Open,High,Low,Close,Volume
Date,,,,,
2018-04-16,1625.38,1632.07,1619.55,1621.21,1.849041e+06
2018-04-17,1617.04,1621.62,1611.19,1616.20,2.091422e+06
2018-04-18,1624.97,1645.43,1623.29,1634.17,2.887990e+06
2018-04-19,1635.00,1648.36,1633.32,1635.41,2.438057e+06
2018-04-20,1624.54,1629.56,1617.46,1625.38,1.876930e+06
...,...,...,...,...,...
2023-04-06,2646.00,2658.50,2630.00,2641.00,2.036500e+06
2023-04-07,2636.50,2643.00,2621.00,2630.50,1.288000e+06
2023-04-10,2642.00,2648.00,2627.00,2633.50,1.158300e+06


丸坊主

In [9]:
#marubozu = ta.CDLMARUBOZU(df["Open"], df["High"], df["Low"], df["Close"])
marubozu = df.ta.CDLMARUBOZU(df["Open"], df["High"], df["Low"], df["Close"])
# チャートに表示するテキスト
df["marubozu_text"] = marubozu.replace({ 100: "買い", -100: "売り", 0: "" })

# チャートに表示するマーカーの位置を算出
df["marubozu_marker"] = (marubozu/100 * df["High"]).abs().replace({ 0: np.nan })

rdf = df[dt.datetime(2021,11,1):dt.datetime(2022,3,31)]
rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

data =  [ go.Candlestick(x=rdf.index, 
                           open=rdf["Open"], high=rdf["High"], 
                           low=rdf["Low"], close=rdf["Close"],   
                           increasing_line_color="red", 
                           decreasing_line_color="gray"),
            # 検出パターン
            go.Scatter(x=rdf.index, 
                       y=rdf["marubozu_marker"], 
                       mode="markers+text", 
                       text=rdf["marubozu_text"],
                       textposition="top center", 
                       name="丸坊主",
                       marker = { "size": 12, "color": "blue", 
                                   "opacity": 0.6 },
                      textfont = {"size": 14, "color": "black" }),         
         ]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()

AttributeError: ignored

寄付坊主 / 大引坊主

In [ ]:
belthold = ta.CDLBELTHOLD(df["Open"], df["High"], df["Low"], df["Close"])
df["belthold_text"] = belthold.replace({ 100: "買い", -100: "売り", 0: "" })
df["belthold_marker"] = (belthold/100 * df["High"]).abs().replace({ 0: np.nan })

rdf = df[dt.datetime(2021,11,1):dt.datetime(2022,3,31)]
rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

data =  [ go.Candlestick(x=rdf.index, 
                           open=rdf["Open"], high=rdf["High"], 
                           low=rdf["Low"], close=rdf["Close"],   
                           increasing_line_color="red", 
                           decreasing_line_color="gray"),
            # 検出パターン
            go.Scatter(x=rdf.index, 
                       y=rdf["belthold_marker"], 
                       mode="markers+text", 
                       text=rdf["belthold_text"],
                       textposition="top center", 
                       name="寄付坊主 / 大引坊主",
                       marker = { "size": 12, "color": "blue", 
                                   "opacity": 0.6 },
                      textfont = {"size": 14, "color": "black" }),         
         ]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()

包み足

In [ ]:
engulfing = ta.CDLENGULFING(df["Open"], df["High"], df["Low"], df["Close"])
df["engulfing_text"] = engulfing.replace({ 100: "買い", -100: "売り", 0: "" })
df["engulfing_marker"] = (engulfing/100 * df["High"]).abs().replace({ 0: np.nan })

rdf = df[dt.datetime(2021,11,1):dt.datetime(2022,3,31)]
rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

data =  [ go.Candlestick(x=rdf.index, 
                           open=rdf["Open"], high=rdf["High"], 
                           low=rdf["Low"], close=rdf["Close"],   
                           increasing_line_color="red", 
                           decreasing_line_color="gray"),
            # 検出パターン
            go.Scatter(x=rdf.index, 
                       y=rdf["engulfing_marker"], 
                       mode="markers+text", 
                       text=rdf["engulfing_text"],
                       textposition="top center", 
                       name="包み足",
                       marker = { "size": 12, "color": "blue", 
                                   "opacity": 0.6 },
                      textfont = {"size": 14, "color": "black" }),         
         ]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()

はらみ足

In [ ]:
harami = ta.CDLHARAMI(df["Open"], df["High"], df["Low"], df["Close"])
df["harami_text"] = harami.replace({ 100: "買い", -100: "売り", 0: "" })
df["harami_marker"] = (harami/100 * df["High"]).abs().replace({ 0: np.nan })

rdf = df[dt.datetime(2021,11,1):dt.datetime(2022,3,31)]
rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

data =  [ go.Candlestick(x=rdf.index, 
                           open=rdf["Open"], high=rdf["High"], 
                           low=rdf["Low"], close=rdf["Close"],   
                           increasing_line_color="red", 
                           decreasing_line_color="gray"),
            # 検出パターン
            go.Scatter(x=rdf.index, 
                       y=rdf["harami_marker"], 
                       mode="markers+text", 
                       text=rdf["harami_text"],
                       textposition="top center", 
                       name="はらみ足",
                       marker = { "size": 12, "color": "blue", 
                                   "opacity": 0.6 },
                      textfont = {"size": 14, "color": "black" }),         
         ]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()

包み上げ / 包み下げ

In [ ]:
outside = ta.CDL3OUTSIDE(df["Open"], df["High"], df["Low"], df["Close"])
df["outside_text"] = outside.replace({ 100: "買い", -100: "売り", 0: "" })
df["outside_marker"] = (outside/100 * df["High"]).abs().replace({ 0: np.nan })

rdf = df[dt.datetime(2021,11,1):dt.datetime(2022,3,31)]
rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

data =  [ go.Candlestick(x=rdf.index, 
                           open=rdf["Open"], high=rdf["High"], 
                           low=rdf["Low"], close=rdf["Close"],   
                           increasing_line_color="red", 
                           decreasing_line_color="gray"),
            # 検出パターン
            go.Scatter(x=rdf.index, 
                       y=rdf["outside_marker"], 
                       mode="markers+text", 
                       text=rdf["outside_text"],
                       textposition="top center", 
                       name="包み上げ / 包み下げ",
                       marker = { "size": 12, "color": "blue", 
                                   "opacity": 0.6 },
                      textfont = {"size": 14, "color": "black" }),         
         ]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()

はらみ上げ / はらみ下げ

In [ ]:
inside = ta.CDL3INSIDE(df["Open"], df["High"], df["Low"], df["Close"])
df["inside_text"] = inside.replace({ 100: "買い", -100: "売り", 0: "" })
df["inside_marker"] = (inside * df["High"] /100).abs().replace({ 0: np.nan })

rdf = df[dt.datetime(2021,11,1):dt.datetime(2022,3,31)]
rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

data =  [ go.Candlestick(x=rdf.index, 
                           open=rdf["Open"], high=rdf["High"], 
                           low=rdf["Low"], close=rdf["Close"],   
                           increasing_line_color="red", 
                           decreasing_line_color="gray"),
            # 検出パターン
            go.Scatter(x=rdf.index, 
                       y=rdf["inside_marker"], 
                       mode="markers+text", 
                       text=rdf["inside_text"],
                       textposition="top center", 
                       name="はらみ上げ / はらみ下げ",
                       marker = { "size": 12, "color": "blue", 
                                   "opacity": 0.6 },
                      textfont = {"size": 14, "color": "black" }),         
         ]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()